# Section 1: Improved Logo Detector

This notebook implements improvements to the baseline model.

## Improvements Implemented:
1. **Larger Model**: YOLOv8s (small) instead of nano
2. **Enhanced Data Augmentation**: More aggressive augmentation
3. **Hyperparameter Tuning**: Optimized learning rate and other parameters
4. **Longer Training**: More epochs with learning rate scheduling
5. **Multi-scale Training**: Variable image sizes during training


In [1]:
# Import necessary libraries
import os
import json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import yaml
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check device (MPS for Mac M1)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load baseline information
with open('baseline_info.json', 'r') as f:
    baseline_info = json.load(f)

print("Baseline metrics:")
print(f"  Val mAP50: {baseline_info['metrics']['val_map50']:.4f}")
print(f"  Test mAP50: {baseline_info['metrics']['test_map50']:.4f}")


Using device: mps
Baseline metrics:
  Val mAP50: 0.9250
  Test mAP50: 0.9417


In [2]:
# Install ultralytics if not already installed
try:
    from ultralytics import YOLO
    print("Ultralytics already installed")
except ImportError:
    print("Installing ultralytics...")
    import subprocess
    subprocess.check_call(["pip", "install", "ultralytics"])
    from ultralytics import YOLO

# Load dataset config
config_path = Path(baseline_info['dataset_config'])
all_classes = baseline_info['classes']

# Initialize improved YOLOv8 model (small version - better capacity)
print("Initializing YOLO11")
#improved_model = YOLO('yolo11n.pt')  
improved_model = YOLO('yolo11n.pt')  

print(f"Model initialized. Device: {device}")
print(f"Number of classes: {len(all_classes)}")


Ultralytics already installed
Initializing YOLO11
Model initialized. Device: mps
Number of classes: 5


## Training with Improvements

### Key Improvements:
1. **Model**: YOLOv8s (small) - 3x more parameters than nano
2. **Augmentation**: Enhanced augmentation (mosaic, mixup, copy-paste)
3. **Learning Rate**: Optimized LR with cosine annealing
4. **Image Size**: Multi-scale training (480-640)
5. **Epochs**: 100 epochs with early stopping
6. **Optimizer**: AdamW with weight decay


In [3]:
# Train improved model with enhanced settings
print("Starting improved model training...")
print(f"Dataset config: {config_path}")

# # Enhanced training parameters
# improved_results = improved_model.train(
#     data=str(config_path),
#     epochs=50,  # More epochs
#     imgsz=640,
#     batch=8,  # Smaller batch for multiscale imgsz train,
#     device=str(device),
#     project='runs/detect',
#     name='improved',
#     exist_ok=True,
#     patience=5,  # More patience for early stopping
#     save=True,
#     plots=True,
#     verbose=True,
#     # Enhanced augmentation
#     #hsv_h=0.015,  # Hue augmentation- removed color augmentation beacuse logos are color invariant
#     hsv_s=0.7,    # Saturation augmentation
#     hsv_v=0.4,    # Value augmentation
#     degrees=10,   # Rotation augmentation
#     translate=0.1,  # Translation
#     scale=0.5,    # Scale augmentation
#     #shear=2.0,    # Shear augmentation
#     perspective=0.0,  # Perspective # removed perspective augmentation because it good for 3d objects less for logos
#     flipud=0.0,   # Vertical flip
#     fliplr=0.5,   # Horizontal flip
#     mosaic=1.0,   # Mosaic augmentation
#     #mixup=0.1,    # Mixup augmentation
#     #copy_paste=0.1,  # Copy-paste augmentation only use it on section 2 for very small datasets
#     # Learning rate and optimizer
#     lr0=0.01,     # Initial learning rate
#     lrf=0.1,      # Final learning rate (lr0 * lrf)
#     momentum=0.937,  # SGD momentum
#     weight_decay=0.0005,  # Weight decay
#     warmup_epochs=3,  # Warmup epochs
#     warmup_momentum=0.8,  # Warmup momentum
#     warmup_bias_lr=0.1,  # Warmup bias LR
#     # Multi-scale training
#     close_mosaic=25,  # Disable mosaic in last N epochs

#     box=9.5,    # Increase weight of box loss (default is usually 7.5)
#     cls=0.5,    # Weight of classification loss (default 0.5)
#     dfl=1.5,    # Weight of distribution focal loss (default 1.5)

# )

improved_results = improved_model.train(
    # Core Data & Device Settings
    data=str(config_path),
    epochs=40,             # Sufficient time for rare classes to converge
    imgsz=640,
    batch=8,               # Safe for 8GB VRAM with multi-scale
    #accumulate=2,          # Simulates batch=16 for gradient stability
    #multi_scale=True,      # Enabled for resolution robustness
    device=str(device),
    patience=15,           # Higher patience to allow minority classes to catch up
    
    # Loss Weights: Tuned for Identity over mere Localization
    box=7.5,    # Increase weight of box loss (default is usually 7.5)
    cls=0.5,    # Weight of classification loss (default 0.5)
    dfl=1.5,    # Weight of distribution focal loss (default 1.5)
    
    # Augmentation: The "Logo-Safe" Suite
    hsv_h=0.015,           # Restored for color robustness
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=3.0,           # Subtle rotation only
    translate=0.1,         # Shift objects to learn position independence
    scale=0.3,             # Reduced to prevent logos from becoming too tiny
    mosaic=1.0,            # High density learning for small objects
    copy_paste=0.3,        # CRITICAL: Helps balance SAP/FedEx by reusing instances
    
    # Explicitly Omitted/Disabled (The "Risky" ones for logos)
    shear=0.0,             # Prevents distortion of circles/squares
    perspective=0.0,       # Prevents unrealistic 3D warping
    flipud=0.0,            # Logos should not be upside down
    #fliplr=0.5,            # Horizontal flips are usually fine
    mixup=0.0,             # Keep features sharp for small objects
    
    # Final Optimization
    close_mosaic=10,       # Disable mosaic in last 10 epochs to sharpen edges
    val=True,              # Ensure validation runs to monitor F1-score
    plots=True             # Generate result plots for final analysis
)
# improved_results = improved_model.train(
#     data=str(config_path),
#     epochs=35,           # Reduced: Your plots show convergence by epoch 20-25
#     imgsz=640,
#     batch=16,             # Lowered for VRAM safety with multi-scale
#     #accumulate=2,        # IMPORTANT: Simulates the batch=16 stability from your baseline
#     multi_scale=True,    # Enabled: Safe now due to batch=8 and accumulate=2
#     device=str(device),
    
#     # Augmentation: Tuned for Logos
#     hsv_h=0.015,         # Restored: Logos need color robustness for lighting shifts
#     hsv_s=0.7,
#     hsv_v=0.4,
#     degrees=2.0,         # Reduced: Logos are rarely tilted 10+ degrees
#     translate=0.1,
#     scale=0.3,           # Reduced: Prevents logos from shrinking into unrecognizable blobs
#     shear=0.0,           # Removed: 2D logos don't need 3D-style shearing
#     perspective=0.0,
#     flipud=0.0,
#     fliplr=0.5,
#     mosaic=1.0,          # Essential for small objects
#     mixup=0.0,           # Disabled: Can make small logos too blurry/ghosted
#     copy_paste=0.0,      # Disabled: Use only if you have <500 images
    
#     # Loss Weights: Balanced for Identity
#     box=7.5,             # Baseline value: Your plots show this is already accurate
#     cls=1.0,             # Increased: Prioritizes identifying the BRAND correctly
#     dfl=1.5,
    
#     # Finishing Move
#     close_mosaic=5,      # Only turn off mosaic at the very end to sharpen edges
#     patience=10          # Slightly more patience to let multi-scale settle
# )
print("\nImproved training completed!")
print(f"Results saved to: {improved_results.save_dir}")


Starting improved model training...
Dataset config: yolo_dataset/dataset.yaml
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.1 MPS (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/dataset.yaml, degrees=3.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

python(2799) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(2800) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

In [9]:
# Load the best improved model
improved_best = YOLO(improved_results.save_dir / 'weights' / 'best.pt')

# Evaluate on validation set
print("Evaluating improved model on validation set...")
val_metrics_improved = improved_best.val(data=str(config_path), split='val')
print(f"\nImproved Validation mAP50: {val_metrics_improved.box.map50:.4f}")
print(f"Improved Validation mAP50-95: {val_metrics_improved.box.map:.4f}")

# Evaluate on test set
print("\nEvaluating improved model on test set...")
test_metrics_improved = improved_best.val(data=str(config_path), split='test')
print(f"\nImproved Test mAP50: {test_metrics_improved.box.map50:.4f}")
print(f"Improved Test mAP50-95: {test_metrics_improved.box.map:.4f}")

# Save metrics
improved_metrics = {
    'val_map50': float(val_metrics_improved.box.map50),
    'val_map50_95': float(val_metrics_improved.box.map),
    'test_map50': float(test_metrics_improved.box.map50),
    'test_map50_95': float(test_metrics_improved.box.map)
}

with open('improved_metrics.json', 'w') as f:
    json.dump(improved_metrics, f, indent=2)

print("\nImproved metrics saved to improved_metrics.json")

# Compare with baseline
print("\n" + "="*60)
print("COMPARISON: Baseline vs Improved")
print("="*60)
print(f"{'Metric':<25} {'Baseline':<15} {'Improved':<15} {'Improvement':<15}")
print("-"*60)
print(f"{'Val mAP50':<25} {baseline_info['metrics']['val_map50']:<15.4f} {improved_metrics['val_map50']:<15.4f} {improved_metrics['val_map50'] - baseline_info['metrics']['val_map50']:+.4f}")
print(f"{'Val mAP50-95':<25} {baseline_info['metrics']['val_map50_95']:<15.4f} {improved_metrics['val_map50_95']:<15.4f} {improved_metrics['val_map50_95'] - baseline_info['metrics']['val_map50_95']:+.4f}")
print(f"{'Test mAP50':<25} {baseline_info['metrics']['test_map50']:<15.4f} {improved_metrics['test_map50']:<15.4f} {improved_metrics['test_map50'] - baseline_info['metrics']['test_map50']:+.4f}")
print(f"{'Test mAP50-95':<25} {baseline_info['metrics']['test_map50_95']:<15.4f} {improved_metrics['test_map50_95']:<15.4f} {improved_metrics['test_map50_95'] - baseline_info['metrics']['test_map50_95']:+.4f}")
print("="*60)


Evaluating improved model on validation set...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.1 CPU (Apple M1)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 822.3±84.3 MB/s, size: 297.4 KB)
val: Scanning /Users/yanivz/Documents/code/sports/logo/logos_detector/yolo_dataset/val/labels.cache... 188 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 188/188 517.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 1.8s/it 21.3s1.9s
                   all        188        458       0.94      0.896      0.948      0.794
                 fedex         39         92      0.941      0.859      0.928      0.753
                  nike         55        108      0.967      0.824      0.913      0.747
                  puma         40        117      0.962      0.866      0.938      0.755
                   sap         27         68      0.

## Visualize Training Results - Improved Model


In [10]:
# Load training results
results_csv = improved_results.save_dir / 'results.csv'
if results_csv.exists():
    import pandas as pd
    results_df = pd.read_csv(results_csv)
    
    # Plot training and validation losses
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Box loss
    axes[0, 0].plot(results_df['epoch'], results_df['train/box_loss'], label='Train Box Loss', linewidth=2)
    axes[0, 0].plot(results_df['epoch'], results_df['val/box_loss'], label='Val Box Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Box Loss')
    axes[0, 0].set_title('Improved Model: Box Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # # Objectness loss
    # axes[0, 1].plot(results_df['epoch'], results_df['train/obj_loss'], label='Train Obj Loss', linewidth=2)
    # axes[0, 1].plot(results_df['epoch'], results_df['val/obj_loss'], label='Val Obj Loss', linewidth=2)
    # axes[0, 1].set_xlabel('Epoch')
    # axes[0, 1].set_ylabel('Objectness Loss')
    # axes[0, 1].set_title('Improved Model: Objectness Loss')
    # axes[0, 1].legend()
    # axes[0, 1].grid(True, alpha=0.3)
    
    # Classification loss
    axes[1, 0].plot(results_df['epoch'], results_df['train/cls_loss'], label='Train Cls Loss', linewidth=2)
    axes[1, 0].plot(results_df['epoch'], results_df['val/cls_loss'], label='Val Cls Loss', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Classification Loss')
    axes[1, 0].set_title('Improved Model: Classification Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # mAP50
    axes[1, 1].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50', linewidth=2, color='green')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('mAP50')
    axes[1, 1].set_title('Improved Model: Validation mAP50')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('improved_training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Training curves saved to improved_training_curves.png")
    
    # Compare training curves with baseline
    baseline_csv = Path(baseline_info['model_path']).parent.parent / 'results.csv'
    if baseline_csv.exists():
        baseline_df = pd.read_csv(baseline_csv)
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Box loss comparison
        axes[0, 0].plot(baseline_df['epoch'], baseline_df['val/box_loss'], 
                       label='Baseline Val Box Loss', linewidth=2, linestyle='--', alpha=0.7)
        axes[0, 0].plot(results_df['epoch'], results_df['val/box_loss'], 
                       label='Improved Val Box Loss', linewidth=2)
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Validation Box Loss')
        axes[0, 0].set_title('Box Loss Comparison')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # mAP50 comparison
        axes[0, 1].plot(baseline_df['epoch'], baseline_df['metrics/mAP50(B)'], 
                        label='Baseline mAP50', linewidth=2, linestyle='--', alpha=0.7)
        axes[0, 1].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], 
                       label='Improved mAP50', linewidth=2)
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('mAP50')
        axes[0, 1].set_title('mAP50 Comparison')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Total loss comparison
        # baseline_total = baseline_df['val/box_loss'] + baseline_df['val/obj_loss'] + baseline_df['val/cls_loss']
        # improved_total = results_df['val/box_loss'] + results_df['val/obj_loss'] + results_df['val/cls_loss']
        
        baseline_total = baseline_df['val/box_loss']  + baseline_df['val/cls_loss']
        improved_total = results_df['val/box_loss']  + results_df['val/cls_loss']

        axes[1, 0].plot(baseline_df['epoch'], baseline_total, 
                       label='Baseline Total Val Loss', linewidth=2, linestyle='--', alpha=0.7)
        axes[1, 0].plot(results_df['epoch'], improved_total, 
                       label='Improved Total Val Loss', linewidth=2)
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Total Validation Loss')
        axes[1, 0].set_title('Total Loss Comparison')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Metrics comparison bar chart
        metrics_comparison = {
            'Val mAP50': [baseline_info['metrics']['val_map50'], improved_metrics['val_map50']],
            'Test mAP50': [baseline_info['metrics']['test_map50'], improved_metrics['test_map50']],
            'Val mAP50-95': [baseline_info['metrics']['val_map50_95'], improved_metrics['val_map50_95']],
            'Test mAP50-95': [baseline_info['metrics']['test_map50_95'], improved_metrics['test_map50_95']]
        }
        
        x = np.arange(len(metrics_comparison))
        width = 0.35
        fig2, ax = plt.subplots(figsize=(12, 6))
        baseline_vals = [v[0] for v in metrics_comparison.values()]
        improved_vals = [v[1] for v in metrics_comparison.values()]
        ax.bar(x - width/2, baseline_vals, width, label='Baseline', alpha=0.8)
        ax.bar(x + width/2, improved_vals, width, label='Improved', alpha=0.8)
        ax.set_ylabel('mAP Score')
        ax.set_title('Model Performance Comparison')
        ax.set_xticks(x)
        ax.set_xticklabels(metrics_comparison.keys())
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("Comparison plots saved to model_comparison.png")
else:
    print("Results CSV not found. Training may still be in progress.")


<Figure size 1600x1200 with 4 Axes>

Training curves saved to improved_training_curves.png


<Figure size 1600x1200 with 4 Axes>

<Figure size 1200x600 with 1 Axes>

Comparison plots saved to model_comparison.png


In [11]:
results_csv = improved_results.save_dir / 'results.csv'

if results_csv.exists():
    import pandas as pd
    results_df = pd.read_csv(results_csv)
    
    # Plot training and validation losses
    #fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fix, axes = plt.subplots(2, 3, figsize=(20, 12))
    # Box loss
    axes[0, 0].plot(results_df['epoch'], results_df['train/box_loss'], label='Train Box Loss', linewidth=2)
    axes[0, 0].plot(results_df['epoch'], results_df['val/box_loss'], label='Val Box Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Box Loss')
    axes[0, 0].set_title('Box Loss: Training vs Validation')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # # Objectness loss
    # axes[0, 1].plot(results_df['epoch'], results_df['train/obj_loss'], label='Train Obj Loss', linewidth=2)
    # axes[0, 1].plot(results_df['epoch'], results_df['val/obj_loss'], label='Val Obj Loss', linewidth=2)
    # axes[0, 1].set_xlabel('Epoch')
    # axes[0, 1].set_ylabel('Objectness Loss')
    # axes[0, 1].set_title('Objectness Loss: Training vs Validation')
    # axes[0, 1].legend()
    # axes[0, 1].grid(True, alpha=0.3)


    # mAP50-95
    axes[1, 1].plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP50-95', linewidth=2, color='green')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('mAP50-95')
    axes[1, 1].set_title('Validation mAP50-95 Over Time')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Classification loss
    axes[0, 1].plot(results_df['epoch'], results_df['train/cls_loss'], label='Train Cls Loss', linewidth=2)
    axes[0, 1].plot(results_df['epoch'], results_df['val/cls_loss'], label='Val Cls Loss', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Classification Loss')
    axes[0, 1].set_title('Classification Loss: Training vs Validation')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    #train/dfl_loss
    axes[0, 2].plot(results_df['epoch'], results_df['train/dfl_loss'], label='Train dfl Loss', linewidth=2)
    axes[0, 2].plot(results_df['epoch'], results_df['val/dfl_loss'], label='Val dfl Loss', linewidth=2)
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('dfl Loss')
    axes[0, 2].set_title('dfl Loss: Training vs Validation')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # mAP50
    axes[1, 0].plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50', linewidth=2, color='green')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('mAP50')
    axes[1, 0].set_title('Validation mAP50 Over Time')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    

    results_df['metrics/F1(B)'] = 2 * (results_df['metrics/precision(B)'] * results_df['metrics/recall(B)']) / \
                                 (results_df['metrics/precision(B)'] + results_df['metrics/recall(B)'] + 1e-6)

    # Add this to your plotting loop or as a new subplot
    axes[1, 2].plot(results_df['epoch'], results_df['metrics/F1(B)'], label='F1 Score', color='orange', linewidth=2)
    axes[1, 2].set_title('Model Reliability (F1 Score)')
    axes[1, 2].legend()

    plt.tight_layout()
    plt.savefig('improved0_training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Training curves saved to improved0_training_curves.png")
else:
    print("Results CSV not found. Training may still be in progress.")

<Figure size 2000x1200 with 6 Axes>

Training curves saved to improved0_training_curves.png


In [12]:
# Run predictions on test images
YOLO_DATASET = Path("yolo_dataset")
test_image_files = list((YOLO_DATASET / 'test' / 'images').glob("*.jpg"))[:12]

# Get predictions
predictions = improved_best.predict(
    source=[str(f) for f in test_image_files],
    conf=0.25,
    iou=0.45,
    save=False,
    show=False
)

# Visualize predictions
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

for idx, (img_file, pred) in enumerate(zip(test_image_files, predictions)):
    # Load original image
    img = Image.open(img_file)
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f"{img_file.name[:40]}...", fontsize=8)
    
    # Draw predictions
    if pred.boxes is not None and len(pred.boxes) > 0:
        boxes = pred.boxes.xyxy.cpu().numpy()
        confidences = pred.boxes.conf.cpu().numpy()
        classes = pred.boxes.cls.cpu().numpy().astype(int)
        
        for box, conf, cls in zip(boxes, confidences, classes):
            x1, y1, x2, y2 = box
            class_name = all_classes[cls]
            
            # Draw bounding box
            from matplotlib.patches import Rectangle
            rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, 
                           edgecolor='cyan', facecolor='none')
            axes[idx].add_patch(rect)
            axes[idx].text(x1, y1-5, f"{class_name} {conf:.2f}", 
                          color='cyan', fontsize=7,
                          bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.7))

plt.tight_layout()
plt.savefig('improved_test_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("Improved test predictions saved to improved_test_predictions.png")



0: 640x640 4 saps, 58.8ms
1: 640x640 1 puma, 58.8ms
2: 640x640 2 nikes, 1 sap, 58.8ms
3: 640x640 2 saps, 58.8ms
4: 640x640 3 saps, 58.8ms
5: 640x640 1 sap, 58.8ms
6: 640x640 1 nike, 1 spotify, 58.8ms
7: 640x640 1 nike, 58.8ms
8: 640x640 1 nike, 1 spotify, 58.8ms
9: 640x640 2 saps, 58.8ms
10: 640x640 1 nike, 58.8ms
11: 640x640 (no detections), 58.8ms
Speed: 1.6ms preprocess, 58.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 2000x1500 with 12 Axes>

Improved test predictions saved to improved_test_predictions.png


In [15]:
# Save improved model info for Section 2
improved_info = {
    'model_path': str(improved_results.save_dir / 'weights' / 'best.pt'),
    'classes': all_classes,
    'num_classes': len(all_classes),
    'metrics': improved_metrics,
    'dataset_config': str(config_path),
    'baseline_metrics': baseline_info['metrics']
}

with open('improved_info.json', 'w') as f:
    json.dump(improved_info, f, indent=2)

print("\nImproved model information saved to improved_info.json")
print("\n" + "="*50)
print("IMPROVED MODEL COMPLETE")
print("="*50)
print(f"Baseline Test mAP50: {baseline_info['metrics']['test_map50']:.4f}")
print(f"Improved Test mAP50: {improved_metrics['test_map50']:.4f}")
print(f"Improvement: {improved_metrics['test_map50'] - baseline_info['metrics']['test_map50']:+.4f} ({((improved_metrics['test_map50'] / baseline_info['metrics']['test_map50'] - 1) * 100):+.2f}%)")



Improved model information saved to improved_info.json

IMPROVED MODEL COMPLETE
Baseline Test mAP50: 0.9417
Improved Test mAP50: 0.9493
Improvement: +0.0077 (+0.81%)


Improved model information saved to improved_info.json
without loss func editing
==================================================
IMPROVED MODEL COMPLETE
==================================================
Baseline Test mAP50: 0.8116
Improved Test mAP50: 0.7517
Improvement: -0.0598 (-7.37%)


Improved model information saved to improved_info.json

==================================================
IMPROVED MODEL COMPLETE
==================================================
Baseline Test mAP50: 0.9417
Improved Test mAP50: 0.9493
Improvement: +0.0077 (+0.81%)
